In [1]:
import numpy as np
import pandas as pd
import requests
import urllib

In [2]:
sightings_df = pd.read_pickle("sightings_merged.pkl")
sightings_df.columns

Index(['Date_Time', 'Shape', 'Duration', 'Summary', 'Posted', 'Detail_Link',
       'Detail_Summary', 'Detail_Summary_nltk', 'Country', 'City', 'State',
       'Location', 'Lat', 'Lon', 'timezone_str', 'Time_UTC', 'sentiment_Vader',
       'sentiment_nltk', 'sentiment_TextBlob', 'objectivity_TextBlob',
       'rel0_2007', 'rel1_2007', 'rel2_2007', 'rel3_2007', 'rel4_2007',
       'rel5_2007', 'par0_2007', 'par1_2007', 'par2_2007', 'par3_2007',
       'rel0_2014', 'rel1_2014', 'rel2_2014', 'rel3_2014', 'rel4_2014',
       'rel5_2014', 'par0_2014', 'par1_2014', 'par2_2014', 'par3_2014',
       'State_num', 'time_num', 'FIPS', 'County'],
      dtype='object')

In [3]:
sightings_df.sort_values("Time_UTC", ascending=True)

,Date_Time,Shape,Duration,Summary,Posted,Detail_Link,Detail_Summary,Detail_Summary_nltk,Country,City,...,rel4_2014,rel5_2014,par0_2014,par1_2014,par2_2014,par3_2014,State_num,time_num,FIPS,County
89170,2006-05-15,Unknown,About 2 Seconds,"Four or five lights, on a seeming spinning obj...",5/15/06,http://www.nuforc.org/webreports/050/S50233.html,"\nFour or five lights, on a seeming spinning o...",four five light seeming spinning objecti drivi...,United States,New Town,...,0.009174,0.042813,0.087156,0.279817,0.255352,0.377676,14,-1.937176,39061,Hamilton
89100,2006-05-15,Other,3-4 min.,"three aligned lights: white, red, white",5/15/06,http://www.nuforc.org/webreports/050/S50315.html,"\nthree aligned lights: white, red, whiteDrivi...",three aligned light white red whitedriving hom...,United States,Williamsburg,...,0.006349,0.073016,0.095238,0.336508,0.171429,0.396825,43,-1.937176,51830,Williamsburg
89103,2006-05-15,Formation,15 seconds,"Fuzzy lights over Naples, FL",5/15/06,http://www.nuforc.org/webreports/050/S50312.html,"\nFuzzy lights over Naples, FLTake this for wh...",fuzzy light naples fltake walking dog around 5...,United States,Naples,...,0.056106,0.089109,0.085809,0.069307,0.603960,0.240924,8,-1.937176,12021,Collier
89110,2006-05-15,Unknown,40 to 45 minutes,Cylinder of Light in the Night Skies.,5/15/06,http://www.nuforc.org/webreports/050/S50293.html,\nCylinder of Light in the Night Skies.While d...,cylinder light night skieswhile driving west r...,United States,Fiskdale,...,0.023102,0.095710,0.046205,0.194719,0.283828,0.475248,17,-1.937176,25027,Worcester
89129,2006-05-15,Triangle,12 seconds,"Crudely ""cloaked"" bomerang object flying do We...",5/15/06,http://www.nuforc.org/webreports/050/S50282.html,"\nCrudely ""cloaked"" bomerang object flying do ...",crudely `` cloaked `` bomerang object flying w...,United States,Waynesboro,...,0.002558,0.066496,0.066496,0.340153,0.309463,0.283887,36,-1.937176,42055,Franklin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2021-04-23,Disk,21 seconds,"Blue UFO comes out of sky over San Leandro, Ca...",4/23/21,http://www.nuforc.org/webreports/162/S162577.html,"\nBlue UFO comes out of sky over San Leandro, ...",blue fo come sky san leandro californian recor...,United States,San Leandro,...,0.057878,0.000000,0.096463,0.250804,0.254019,0.398714,3,1.671080,06001,Alameda
87,2021-04-23,Chevron,4 -6 seconds,A soft white couple of opposing chevrons flew ...,4/23/21,http://www.nuforc.org/webreports/162/S162782.html,\nA soft white couple of opposing chevrons fle...,soft white couple opposing chevron flew silent...,United States,Tucson,...,0.003226,0.129032,0.109677,0.219355,0.161290,0.509677,2,1.671080,04019,Pima
155,2021-04-23,Light,3-5 seconds,"We saw a huge, green light flash, then move ac...",4/23/21,http://www.nuforc.org/webreports/162/S162634.html,"\nWe saw a huge, green light flash, then move ...",saw huge green light flash move across sky dis...,United States,Arlington,...,0.088235,0.000000,0.084967,0.169935,0.303922,0.441176,45,1.671080,53061,Snohomish
136,2021-04-23,Sphere,10 minutes,It was over the Cascades and a round ship with...,4/23/21,http://www.nuforc.org/webreports/162/S162581.html,\nIt was over the Cascades and a round ship wi...,cascade round ship spark shooting i4 crest sou...,United States,Sisters,...,0.015018,0.074205,0.074205,0.263251,0.258834,0.403710,35,1.671080,41017,Deschutes


In [4]:
# start_date = '01/01/2015'
# end_date = '12/31/2017'
start_date = '01/01/1900'
end_date = '12/31/2022'
len(sightings_df[(sightings_df["Time_UTC"]>start_date)&(sightings_df["Time_UTC"]<end_date)])

60412

In [5]:
# agg_data = sightings_df[(sightings_df["Time_UTC"]>start_date)&(sightings_df["Time_UTC"]<end_date)].value_counts("FIPS")
agg_data = sightings_df.value_counts("FIPS")
sightings = agg_data.values
graph_fips = agg_data.index
data_df = pd.DataFrame({"fips": graph_fips, "sightings": sightings})
data_df["fips"] = data_df.fips.astype(str)

In [6]:
len(data_df)

2670

In [7]:
import json

# population by county from:
# https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html
# https://api.census.gov/data/2019/pep/population?get=NAME,POP&for=county:*
f = open("resources/county_population.json",)

data = pd.DataFrame(json.load(f))
data.columns = ["name", "population", "state", "county"]
county_population_data = data.iloc[1: , :].copy()
county_population_data["fips"] = county_population_data.state + county_population_data.county
county_population_data.head(5)

,name,population,state,county,fips
1,"Hidalgo County, New Mexico",4198,35,023,35023
2,"Hudson County, New Jersey",672391,34,017,34017
3,"Hunterdon County, New Jersey",124371,34,019,34019
4,"Morris County, New Jersey",491845,34,027,34027
5,"Grant County, New Mexico",26998,35,017,35017


In [8]:
def get_population(fips):
    fips = str(fips).zfill(5)
    p = county_population_data[county_population_data.fips==fips].population
    if len(p) != 1:
        print(f"{fips} not found")
        return None
    
    return p.values[0]

data_df["population"] = data_df["fips"].apply(get_population)
data_df.dropna(axis=0, how="any", inplace=True)
data_df["population"] = data_df["population"].astype(str).astype(int)
data_df["per_cap"] = (data_df["sightings"]/data_df["population"])*1000

66010 not found
78010 not found
78030 not found


In [9]:
no_sightings = county_population_data[county_population_data.fips.isin(data_df.fips)==False].copy()
no_sightings.drop(["name", "state", "county"], axis=1, inplace=True)
no_sightings["per_cap"] = 0
no_sightings["sightings"] = 0

In [10]:
print(f"{len(no_sightings)}, {len(data_df)}")
data_df = data_df.append(no_sightings, ignore_index=True)
print(f"{len(data_df)}")

553, 2667
3220


In [11]:
import math

def calc_log(c):
    try:
        if c > 0:
            return math.log(c)
    except:
        print(c)
    return None

data_df["per_cap_adj"] = data_df["per_cap"].apply(calc_log)

In [12]:
math.log(data_df[data_df.per_cap!=0].per_cap.max()), math.log(data_df[data_df.per_cap!=0].per_cap.min())

(3.499886758502959, -6.564001496131926)

In [13]:
min_val = data_df[data_df.per_cap>0].per_cap.min()
max_val = data_df[data_df.fips!='48283'].per_cap.max()
min_val, max_val

(0.001410231369609655, 8.679826403471932)

In [14]:
offset = .000001
"""
Counties with no sightings do not show up in this aggregated data. They
were added to the dataset but with a per capita value of 0. However, to
smooth the per capita per 1000 people values for the map, the log of the
value was used. The log of 0 is problematic, so a value just below the
lowest adjusted value in the dataset is used.
"""
data_df.loc[data_df.per_cap==0, ['per_cap_adj']] = math.log(min_val-offset)

"""
Adjusting La Salle County in Texas. Its original per capita per 1000 people
value is just over 21, which is nearly four times the next closest value
of 5.8. The purpose is not to diminish this value as "false" or any such
implication. The purpose of the adjustment is so it does not wash out the
other "high" sighting counties.

https://www.bestplaces.net/people/county/texas/la_salle
"""
adjust_la_salle = False
if adjust_la_salle:
    data_df.loc[data_df.fips=='48283', ['per_cap']] = max_val+offset
    data_df.loc[data_df.fips=='48283', ['per_cap_adj']] = math.log(max_val+offset)

In [15]:
math.log(min_val-offset)

-6.564710851164728

In [16]:
min_adj_val = math.log(min_val-offset)
data_df['per_cap_adj'] = data_df['per_cap_adj'] + abs(min_adj_val)

In [17]:
data_df.sort_values("per_cap", ascending=False).head(5)

,fips,sightings,population,per_cap,per_cap_adj
24,48283,249,7520,33.111702,10.064598
92,28147,124,14286,8.679826,8.725712
1210,30037,7,821,8.526188,8.707853
1479,08079,5,769,6.501951,8.436813
915,13061,12,2834,4.234298,8.007928


In [18]:
data_df.sort_values("per_cap_adj", ascending=False).head(5)

,fips,sightings,population,per_cap,per_cap_adj
24,48283,249,7520,33.111702,10.064598
92,28147,124,14286,8.679826,8.725712
1210,30037,7,821,8.526188,8.707853
1479,08079,5,769,6.501951,8.436813
915,13061,12,2834,4.234298,8.007928


In [19]:
data_df.sort_values("sightings", ascending=False).head(5)

,fips,sightings,population,per_cap,per_cap_adj
0,04013,979,4485414,0.218263,5.042656
1,06037,877,10039107,0.087358,4.126974
2,06073,752,3338330,0.225262,5.074221
3,06059,698,3175692,0.219795,5.049649
4,53033,643,2252782,0.285425,5.310934


In [20]:
len(data_df)

3220

In [21]:
data_df.to_pickle("county_sightings.pkl")

In [22]:
agg_data = sightings_df.value_counts("State")
sightings = agg_data.values
graph_fips = agg_data.index
state_sightings = pd.DataFrame({"State": graph_fips, "sightings": sightings})

state_features = ['rel0_2007', 'rel1_2007', 'rel2_2007', 'rel3_2007', 'rel4_2007',
       'rel5_2007', 'par0_2007', 'par1_2007', 'par2_2007', 'par3_2007',
       'rel0_2014', 'rel1_2014', 'rel2_2014', 'rel3_2014', 'rel4_2014',
       'rel5_2014', 'par0_2014', 'par1_2014', 'par2_2014', 'par3_2014']
for state in state_sightings.State.unique():
    state_sightings.loc[state_sightings.State==state, state_features] = sightings_df.loc[sightings_df.State==state, state_features].head(1).values

In [23]:
state_sightings.to_pickle("state_sightings.pkl")